# Neste notebook estão implementadas as funções que determinam as configurações dos Kits por meio do K-Means

In [1]:
from sklearn.cluster import KMeans
from functools import reduce
import pandas as pd
import numpy as np
import math
import re

In [2]:
path = '/home/marcussilva/Documents/bases/'

### Preparando os dados

In [3]:
dt = pd.read_csv(path + 'dadosGenéricos.csv')

In [4]:
dt.head()

,CD_CONVENIO,NM_CONVENIO,CD_AVISO_CIRURGIA,CD_CIRURGIA_AVISO,NOVO_CD_CIRURGIA,DS_CIRURGIA,SN_PRINCIPAL,CD_CID,DS_CID,NOVO_CD_ESPECIALIDADE,...,NOVO_CD_ESPECIE_MESTRE,DS_ESPECIE_MESTRE,NOVO_CD_CLASSE_MESTRE,DS_CLASSE_MESTRE,NOVO_CD_SUB_CLA_MESTRE,DS_SUB_CLA_MESTRE,DT_REALIZACAO,QT_MOVIMENTACAO,ANO_REALIZACAO,ANO_MES_REALIZACAO
0,COV379,Convenio 0,AVC776,CAV731,NCIR412,Cirurgia 0,S,CID994,CID 0,ESP371,...,ESPM584,Especie Mestre 0,CLAM988,Classe Mestre 0,SUBCLAM925,Subclasse Mestre 0,2020-08-01,47,2024,2024-01
1,COV839,Convenio 1,AVC517,CAV970,NCIR382,Cirurgia 1,N,CID787,CID 1,ESP885,...,ESPM375,Especie Mestre 1,CLAM629,Classe Mestre 1,SUBCLAM933,Subclasse Mestre 1,2020-07-13,9,2020,2024-05
2,COV828,Convenio 2,AVC618,CAV197,NCIR866,Cirurgia 2,S,CID549,CID 2,ESP522,...,ESPM206,Especie Mestre 2,CLAM158,Classe Mestre 2,SUBCLAM356,Subclasse Mestre 2,2022-01-11,23,2022,2022-04
3,COV536,Convenio 3,AVC669,CAV467,NCIR744,Cirurgia 3,N,CID511,CID 3,ESP187,...,ESPM214,Especie Mestre 3,CLAM380,Classe Mestre 3,SUBCLAM179,Subclasse Mestre 3,2023-06-27,85,2022,2024-07
4,COV607,Convenio 4,AVC435,CAV559,NCIR556,Cirurgia 4,S,CID618,CID 4,ESP219,...,ESPM116,Especie Mestre 4,CLAM249,Classe Mestre 4,SUBCLAM779,Subclasse Mestre 4,2020-03-12,22,2020,2022-01


In [6]:
dt.dropna(subset= 'DS_CID', inplace=True)

### Algoritmo para selecionar as colunas utilizadas para fazer a montagem dos Kits bem como calcular as frequências de utilização dos itens, o qual é importante no momento da elaboração do Kit

In [7]:
columns = ['DS_CIRURGIA', 'DS_PRODUTO_MESTRE', 'CD_AVISO_CIRURGIA','DS_ESPECIE', 'DS_CLASSE']
df = dt[columns].groupby(['DS_CIRURGIA', 'DS_PRODUTO_MESTRE','DS_ESPECIE', 'DS_CLASSE']).agg(
    freq_m = pd.NamedAgg(column = 'CD_AVISO_CIRURGIA', aggfunc = pd.Series.nunique)).sort_values(by='freq_m', ascending = False).reset_index()

freqs = dt.groupby('DS_CIRURGIA')['CD_AVISO_CIRURGIA'].nunique()
freqs_dict = freqs.to_dict()

columns = ['DS_CIRURGIA', 'NOVO_CD_CIRURGIA','NOVO_CD_PRODUTO_MESTRE','DS_PRODUTO_MESTRE','DS_CID','DS_PRODUTO',
           'DS_UNIDADE_REFERENCIA','DS_ESPECIALID','CD_AVISO_CIRURGIA','QT_MOVIMENTACAO', 'DS_CLASSE', 'ANO_MES_REALIZACAO']
dt_selected = dt[columns]

agg_funcs = {
    'QT_MOVIMENTACAO': ['min', 'max', 'mean', 'median', 'std'],
    'CD_AVISO_CIRURGIA': pd.Series.nunique
}
statistics = dt_selected.groupby(['DS_CIRURGIA', 'NOVO_CD_CIRURGIA','CD_AVISO_CIRURGIA','NOVO_CD_PRODUTO_MESTRE',
                                  'DS_CID','DS_PRODUTO_MESTRE','DS_PRODUTO','DS_UNIDADE_REFERENCIA', 'DS_ESPECIALID', 'ANO_MES_REALIZACAO']).agg(agg_funcs)
statistics.columns = ['min', 'max', 'mean', 'median', 'std', 'freq_prod']  
statistics = statistics.sort_values(by='freq_prod', ascending=False).reset_index()
statistics['freq_prod%'] = statistics['freq_prod'] / statistics['DS_CIRURGIA'].map(freqs_dict)
statistics['srd'] = statistics['std'].fillna(0)
statistics['qt'] = statistics['median'] + round(statistics['std'])
statistics['qt'] = statistics[['max', 'qt']].min(axis=1)

aux = pd.merge(statistics, df, on=['DS_CIRURGIA', 'DS_PRODUTO_MESTRE'], how='left')
aux['freq_m%'] = aux['freq_m'] / aux['DS_CIRURGIA'].map(freqs_dict)
aux.sort_values(by='freq_m%', inplace=True)

## Funções para encontrar o Kit Base

#### Funções para verificar similaridade entre Produtos Mestre e Classes

In [29]:
def similarityProd(df, cid):

    tmp = df.loc[(df['DS_CID'] == cid), ['DS_PRODUTO_MESTRE']]
    k1 = tmp['DS_PRODUTO_MESTRE'].nunique()

    k2 = df['DS_PRODUTO_MESTRE'].nunique()

    inter = pd.merge(df, tmp, how='inner', on= 'DS_PRODUTO_MESTRE')
    intersection = inter['DS_PRODUTO_MESTRE'].nunique()

    if k1 > k2:
        result = intersection/k1
        return result
    else:
        result = intersection/k2
        return result

In [30]:
def similarityClass(df, cid):

    tmp = df.loc[(df['DS_CID'] == cid), ['DS_CLASSE']]
    k1 = tmp['DS_CLASSE'].nunique()

    k2 = df['DS_CLASSE'].nunique()

    inter = pd.merge(df, tmp, how='inner', on= 'DS_CLASSE')
    intersection = inter['DS_CLASSE'].nunique()

    if k1 > k2:
        result = intersection/k1
        return result
    else:
        result = intersection/k2
        return result

#### Funções que geram os pontos, fazem o agrupamento e geram os dados para o K-Means

In [31]:
def kmeansData(df):
    data = {
        "kitCid": [],
        "prodSimilarity": [],
        "classSimilarity": [],
    }

    columnCID = df['DS_CID'].drop_duplicates()
    colunaCID = columnCID.to_numpy()

    dt = pd.DataFrame(data)

    for i in range(len(df['DS_CID'].unique())):
        cid = colunaCID[i]
        x = similarityProd(df, cid)
        y = similarityClass(df, cid)

        dt.loc[i] = [cid, x, y]

    return dt

In [32]:
def kmeansGraph(df):
    data = kmeansData(df)

    inertias = []
    wcss = []
    distances = []

    for k in range(2,10):
        kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
        kmeans.fit(data[['prodSimilarity','classSimilarity']])

        wcss.append(kmeans.inertia_)

    x1, y1 = 2, wcss[0]
    x2, y2 = 10, wcss[len(wcss) - 1]

    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominator = math.sqrt((y2 -y1)**2 + (x2 - x1)**2)

        distances.append(numerator/denominator)

    numCluster = distances.index(max(distances)) + 2

    kmeans = KMeans(n_clusters=numCluster, n_init='auto', random_state=42)
    kmeans.fit(data[['prodSimilarity','classSimilarity']])

    data['cidGrupo'] = kmeans.labels_

    fig = px.scatter(data, x = 'prodSimilarity', y = 'classSimilarity', color='cidGrupo', hover_data={'cidGrupo':True, 'kitCid':True})

    return(fig.show())

In [33]:
def kmeansGraphPlot(df):
    data = kmeansData(df)

    fig = px.scatter(data, x = 'prodSimilarity', y = 'classSimilarity')

    return(fig.show())

#### Funções para fazer as análises dos textos e retornar parte das quantidades dos itens

##### Essas funções funcionam da seguinte forma, dado um medicamento, por exemplo,presente na lista de itens do kit, é extraído a quantidade descrita nele e multiplica-se pela quantidade de frascos que seriam necessários daquele item e tem-se a quantidade total, exemplificando, poderia ter na lista "dipirona 1mg/ml 5ml" sendo preciso 3 frascos, então para aquela situação utiliza-se 15ml, isso foi feito porque no momento da montagem do kit nem sempre estarão disponíveis apenas itens de embalagens de tamanhos iguais, então com a informação do total torna-se mais fácil

In [34]:
def geraTuplas(produto):

  formato = r'(\d+(\.\d+)?)\s*([A-Za-z]+)(?:/([A-Za-z]+))?'
  a = re.findall(formato, produto)

  output = [(valor, unidade1+'/'+unidade2 if unidade2 else unidade1)
          for valor, _, unidade1, unidade2 in a]
  return output

In [35]:
def geraQuantidades(df):
    df['DS_PRODUTO_MESTRE'] = df['DS_PRODUTO_MESTRE'].apply(lambda x: re.sub(r'(\d+),(\d+)', r'\1.\2', x))

    df['qt_parcial'] = 1
    df['unidade_texto'] = df['unit']

    for idx, row in df.iterrows():
        if row['DS_ESPECIE'] == 'MATERIAIS HOSPITALARES':
            continue

        produto = row['DS_PRODUTO_MESTRE']
        vet = geraTuplas(produto)

        if not vet:
            continue

        vet_filtrado = [tup for tup in vet if '/' not in tup[1]]
        if len(vet_filtrado) == 1:
            valor, unit = vet_filtrado[0]
            df.at[idx, 'qt_parcial'] = float(valor)
            df.at[idx, 'unidade_texto'] = unit
    
    df['qt_parcial'] = df['qt_parcial'].astype(float)
    df['Quant_final'] = df['qt_parcial'] * df['quant']

    df = df.iloc[:, [0, 1, 2, 3, 4, 6, 5]]

    return df

#### Funções para definir os kits base,ou seja, os kits finais

##### Essas funções utilizam do algoritmo de agrupamento K-Means para montar os grupos de doenças que possuem similaridade com relação a configuração dos seus Kits, feito os grupos, cada um deles representará um kit para aquele procedimento cirúrgico

In [36]:
def baseKit(df, list):
    i = 0
    dataframes = []
    v = list['kitCid'].drop_duplicates()
    v = v.to_numpy()

    while(i < len(v)):
        aux = v[i]

        dt = df.loc[(df['DS_CID'] == aux), ['DS_PRODUTO_MESTRE']]
        dataframes.append(dt)
        i += 1

    L = [pd.DataFrame(np.sort(x.values, axis=1), columns=x.columns).drop_duplicates() for x in dataframes]
    dfBase = reduce(lambda left, right: pd.merge(left,right), L)

    prev_kit = pd.merge(df, dfBase, how='inner', on = 'DS_PRODUTO_MESTRE')

    prev_kit = prev_kit.groupby(['DS_PRODUTO_MESTRE', 'DS_UNIDADE_REFERENCIA', 'DS_ESPECIE'])['qt'].min().reset_index()
    prev_kit['Qt'] = prev_kit['qt']

    kit = {'DS_PRODUTO_MESTRE': prev_kit['DS_PRODUTO_MESTRE'].to_list(),
           'quant': np.around(prev_kit['Qt'],1).to_list(),
           'unit': prev_kit['DS_UNIDADE_REFERENCIA'].to_list(),
           'DS_ESPECIE': prev_kit['DS_ESPECIE'].to_list()}

    kit = pd.DataFrame(kit)

    kit = geraQuantidades(kit)

    return kit

In [37]:
def finalKit(df, cid):
    data = kmeansData(df)

    wcss = []
    distances = []

    for k in range(2,10):
        kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
        kmeans.fit(data[['prodSimilarity','classSimilarity']])

        wcss.append(kmeans.inertia_)

    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss) - 1]

    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominator = math.sqrt((y2 -y1)**2 + (x2 - x1)**2)

        distances.append(numerator/denominator)

    numCluster = distances.index(max(distances)) + 2

    kmeans = KMeans(n_clusters=numCluster, n_init='auto', random_state=42)
    kmeans.fit(data[['prodSimilarity','classSimilarity']])

    data['cidGrupo'] = kmeans.labels_

    tmp1 = data.loc[(data['kitCid'] == cid)]
    tmp2 = tmp1['cidGrupo']

    list = data.loc[(data['cidGrupo'] == tmp2.iloc[0])]

    res = baseKit(df, list)
    data['cidGrupo'] = data['cidGrupo'].astype(str)
    fig = px.scatter(data, x='prodSimilarity', y='classSimilarity', color='cidGrupo',
                 hover_data={'cidGrupo': True, 'kitCid': True},
                 category_orders={'cidGrupo': data['cidGrupo'].unique().tolist()})
    fig.update_layout(legend_title_font_size=18, legend_font_size=18)
    fig.show()

    return res

## Invertendo o CID pelo procedimento cirúrgico

### Essas funções são as mesmas das descritas anteriormente, a única diferença é a troca do procedimento pelo CID, e vice versa

In [ ]:
def similarityProdCid(df, cir):

    tmp = df.loc[(df['DS_CIRURGIA'] == cir), ['DS_PRODUTO_MESTRE']]
    k1 = tmp['DS_PRODUTO_MESTRE'].nunique()

    k2 = df['DS_PRODUTO_MESTRE'].nunique()

    inter = pd.merge(df, tmp, how='inner', on= 'DS_PRODUTO_MESTRE')
    intersection = inter['DS_PRODUTO_MESTRE'].nunique()

    if k1 > k2:
        result = intersection/k1
        return result
    else:
        result = intersection/k2
        return result

In [ ]:
def similarityClassCid(df, cir):

    tmp = df.loc[(df['DS_CIRURGIA'] == cir), ['DS_CLASSE']]
    k1 = tmp['DS_CLASSE'].nunique()

    k2 = df['DS_CLASSE'].nunique()

    inter = pd.merge(df, tmp, how='inner', on= 'DS_CLASSE')
    intersection = inter['DS_CLASSE'].nunique()

    if k1 > k2:
        result = intersection/k1
        return result
    else:
        result = intersection/k2
        return result

In [ ]:
def kmeansDataCid(df):
    data = {
        "kitCir": [],
        "prodSimilarity": [],
        "classSimilarity": [],
    }

    columnCIR = df['DS_CIRURGIA'].drop_duplicates()
    colunaCIR = columnCIR.to_numpy()

    dt = pd.DataFrame(data)

    for i in range(len(df['DS_CIRURGIA'].unique())):
        cir = colunaCIR[i]
        x = similarityProdCid(df, cir)
        y = similarityClassCid(df, cir)

        dt.loc[i] = [cir, x, y]

    return dt

In [ ]:
def kmeansGraphCid(df):
    data = kmeansDataCid(df)

    wcss = []
    distances = []

    for k in range(2,10):
        kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
        kmeans.fit(data[['prodSimilarity','classSimilarity']])

        wcss.append(kmeans.inertia_)

    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss) - 1]

    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominator = math.sqrt((y2 -y1)**2 + (x2 - x1)**2)

        distances.append(numerator/denominator)

    numCluster = distances.index(max(distances)) + 2

    kmeans = KMeans(n_clusters=numCluster, n_init='auto')
    kmeans.fit(data[['prodSimilarity','classSimilarity']])

    data['cirGrupo'] = kmeans.labels_

    fig = px.scatter(data, x = 'prodSimilarity', y = 'classSimilarity',
                     color='cirGrupo',
                     hover_data={'cirGrupo':True, 'kitCir':True})

    return(fig.show())

In [ ]:
def baseKitCid(df, list):
    i = 0
    dataframes = []
    v = list['kitCir'].drop_duplicates()
    v = v.to_numpy()

    while(i < len(v)):
        aux = v[i]

        dt = df.loc[(df['DS_CIRURGIA'] == aux), ['DS_PRODUTO_MESTRE']]
        dataframes.append(dt)
        i += 1

    L = [pd.DataFrame(np.sort(x.values, axis=1), columns=x.columns
                      ).drop_duplicates() for x in dataframes]
    dfBase = reduce(lambda left, right: pd.merge(left,right), L)

    kit = dfBase['DS_PRODUTO_MESTRE']
    kit = kit.to_numpy()

    return kit

In [ ]:
def finalKitCid(df, cid):
    data = kmeansDataCid(df)

    wcss = []
    distances = []

    for k in range(2,10):
        kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
        kmeans.fit(data[['prodSimilarity','classSimilarity']])

        wcss.append(kmeans.inertia_)

    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss) - 1]

    for i in range(len(wcss)):
        x0 = i + 2
        y0 = wcss[i]
        numerator = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
        denominator = math.sqrt((y2 -y1)**2 + (x2 - x1)**2)

        distances.append(numerator/denominator)

    numCluster = distances.index(max(distances)) + 2

    kmeans = KMeans(n_clusters=numCluster, n_init='auto', random_state=42)
    kmeans.fit(data[['prodSimilarity','classSimilarity']])

    data['cirGrupo'] = kmeans.labels_

    tmp1 = data.loc[(data['kitCir'] == cid)]
    tmp2 = tmp1['cirGrupo']

    list = data.loc[(data['cirGrupo'] == tmp2.iloc[0])]

    res = baseKitCid(df, list)

    fig = px.scatter(data, x = 'prodSimilarity', y = 'classSimilarity', color='cirGrupo', hover_data={'cirGrupo':True, 'kitCir':True})
    fig.show()

    print(res)

In [ ]:
a = aux['DS_CID'] == 'DOR AGUDA'

df = aux.loc[(a)&(aux['freq_m%']>0.1), ['DS_CIRURGIA', 'DS_CLASSE', 'DS_PRODUTO_MESTRE']]

kmeansGraphCid(df)